# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fba60294130>
├── 'a' --> tensor([[-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452]])
└── 'x' --> <FastTreeValue 0x7fb98896f9d0>
    └── 'c' --> tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                        [-0.7522,  0.6208, -1.8580, -0.2801],
                        [ 0.2884, -0.3771,  1.6102, -0.1751]])

In [4]:
t.a

tensor([[-1.0393, -1.1451, -1.3067],
        [-0.9535, -0.3275,  0.0452]])

In [5]:
%timeit t.a

85.9 ns ± 0.6 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fba60294130>
├── 'a' --> tensor([[ 0.0715, -0.3541, -2.5739],
│                   [ 1.4846, -0.0583,  0.3116]])
└── 'x' --> <FastTreeValue 0x7fb98896f9d0>
    └── 'c' --> tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                        [-0.7522,  0.6208, -1.8580, -0.2801],
                        [ 0.2884, -0.3771,  1.6102, -0.1751]])

In [7]:
%timeit t.a = new_value

91.9 ns ± 0.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452]]),
    x: Batch(
           c: tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                      [-0.7522,  0.6208, -1.8580, -0.2801],
                      [ 0.2884, -0.3771,  1.6102, -0.1751]]),
       ),
)

In [10]:
b.a

tensor([[-1.0393, -1.1451, -1.3067],
        [-0.9535, -0.3275,  0.0452]])

In [11]:
%timeit b.a

78.9 ns ± 0.244 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1510,  0.5349,  0.3226],
               [ 0.4907,  1.1867, -1.3963]]),
    x: Batch(
           c: tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                      [-0.7522,  0.6208, -1.8580, -0.2801],
                      [ 0.2884, -0.3771,  1.6102, -0.1751]]),
       ),
)

In [13]:
%timeit b.a = new_value

653 ns ± 3.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 9.67 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 93.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

174 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

178 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fba602e4a60>
├── 'a' --> tensor([[[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]],
│           
│                   [[-1.0393, -1.1451, -1.3067],
│                    [-0.9535, -0.3275,  0.0452]]])
└── 'x' --> <FastTreeValue 0x7fba602e4a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.1 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb9887a6e20>
├── 'a' --> tensor([[-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452],
│                   [-1.0393, -1.1451, -1.3067],
│                   [-0.9535, -0.3275,  0.0452]])
└── 'x' --> <FastTreeValue 0x7fb9803dc4c0>
    └── 'c' --> tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                        [-0.7522,  0.6208, -1.8580, -0.2801],
                 

In [23]:
%timeit t_cat(trees)

37.3 µs ± 702 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.4 µs ± 595 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]],
       
               [[-1.0393, -1.1451, -1.3067],
                [-0.9535, -0.3275,  0.0452]]]),
    x: Batch(
           c: tensor([[[ 0.8386,  0.7699, -0.3062, -0.2319],
                       [-0.7522,  0.6208, -1.8580, -0.2801],
                       [ 0.2884, -0.3771,  1.6102, -0.1751]],
         

In [26]:
%timeit Batch.stack(batches)

98.6 µs ± 579 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452],
               [-1.0393, -1.1451, -1.3067],
               [-0.9535, -0.3275,  0.0452]]),
    x: Batch(
           c: tensor([[ 0.8386,  0.7699, -0.3062, -0.2319],
                      [-0.7522,  0.6208, -1.8580, -0.2801],
                      [ 0.2884, -0.3771,  1.6102, -0.1751],
                      [ 0.8386,  0.7699, -0.3062, -0.2319],
                      [-0.7522,  

In [28]:
%timeit Batch.cat(batches)

181 µs ± 673 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

448 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
